In [13]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

tqdm.pandas()

In [14]:
path = 'google_scholar/data'
root_folder = Path(path)
model_name = 'qwen3_8b'
df_searchterms = pd.read_csv(path+'/google_scholar_search_base_article_detail_txt.csv',sep =';')
df_searchterms['output_folder'] = path +'/score/'+ model_name+'/'+df_searchterms['folder']
df_searchterms['output_file'] =  df_searchterms['output_folder']+'/'+df_searchterms['article_file'].str.replace('.html','.txt').str.replace('.pdf','.txt')


In [15]:
files = list((root_folder/'score/qwen3_8b' ).rglob("*"))  
files = [f for f in files if f.is_file() and f.suffix in (".txt")]
lst_name_files = [file.name for file in files]
s = df_searchterms['article_file'].str.replace('.html','.txt').str.replace('.pdf','.txt') 
df_searchterms = df_searchterms[s.isin(lst_name_files)].copy()

In [16]:
def load_file(path):
    f= open(path, "r", encoding="utf-8") 
    document = f.read()
    f.close()
    return document

df_searchterms['file_score'] = df_searchterms['output_file'].progress_map(load_file)

  0%|          | 0/2468 [00:00<?, ?it/s]

In [17]:
df_searchterms['think'] = df_searchterms['file_score'].str.extract('<think>(.*?)</think>')
df_searchterms['file_score_limpo'] = df_searchterms['file_score'].str.split('</think>').map(lambda x : x[-1])
df_searchterms['abstract'] = df_searchterms['file_score_limpo'].str.extract('<ABSTRACT>(.*?)</ABSTRACT>')
df_searchterms['assessment'] = df_searchterms['file_score_limpo'].str.extract('<RELEVANCE ASSESSMENT>(.*?)</RELEVANCE ASSESSMENT>')
df_searchterms['score'] = df_searchterms['file_score_limpo'].str.extract('<SCORE>(.*?)</SCORE>')

In [18]:
def get_score(x):
    try:
        return int(x)
    except:
        return None

df_searchterms['score'] = df_searchterms['score'].map(get_score)

In [19]:
px.histogram(df_searchterms['score'])

In [20]:
df_searchterms[df_searchterms['score']>=80].shape[0]

228

In [78]:
df_searchterms[df_searchterms['score']>=80]['href'].values[5]

'https://link.springer.com/chapter/10.1007/978-3-030-65854-0_3'

In [87]:
df_searchterms[df_searchterms['score']>=80]

,searchterm,start,folder,url,file,path_file,href,title,author_div,description,...,path_article_file,path_txt_folder,path_txt_article,folder_score,file_score,think,file_score_limpo,abstract,assessment,score
7,visualization machine learning,70,visualization_machine_learning,https://scholar.google.com.br/scholar?start=70...,https_scholar_google_com_br_scholar_start_70_q...,data/raw_data/visualization_machine_learning/h...,https://www.sciencedirect.com/science/article/...,Visualization and characterization of agricult...,"MS Kumar, CJ Hogan, SA Fredericks, J Hong - Co...",Accurate characterization of agricultural spra...,...,data/articles/visualization_machine_learning/h...,data/articles_txt/visualization_machine_learning,data/articles_txt/visualization_machine_learni...,google_scholar/data/score/visualization_machin...,"<think>\nOkay, let's tackle this query. The us...",NaN,\n\n<ABSTRACT>The article presents a system fo...,The article presents a system for monitoring m...,"Yes, the article directly addresses model moni...",96.0
18,visual machine learning,0,visual_machine_learning,https://scholar.google.com.br/scholar?start=0&...,https_scholar_google_com_br_scholar_start_0_q_...,data/raw_data/visual_machine_learning/https_sc...,https://onlinelibrary.wiley.com/doi/abs/10.111...,The state of the art in integrating machine le...,"A Endert, W Ribarsky, C Turkay… - Computer gra...",… We claim that visual analytics (VA) and mach...,...,data/articles/visual_machine_learning/https_ar...,data/articles_txt/visual_machine_learning,data/articles_txt/visual_machine_learning/http...,google_scholar/data/score/visual_machine_learning,"<think>\nOkay, let's tackle this query. The us...",NaN,\n\n<ABSTRACT>The article explores frameworks ...,The article explores frameworks for integratin...,"Yes, the article directly addresses model obse...",92.0
22,visual machine learning,20,visual_machine_learning,https://scholar.google.com.br/scholar?start=20...,https_scholar_google_com_br_scholar_start_20_q...,data/raw_data/visual_machine_learning/https_sc...,https://ieeexplore.ieee.org/abstract/document/...,explAIner: A visual analytics framework for in...,"T Spinner, U Schlegel, H Schäfer… - IEEE trans...","… , a visual analytics system for interactive ...",...,data/articles/visual_machine_learning/https_ar...,data/articles_txt/visual_machine_learning,data/articles_txt/visual_machine_learning/http...,google_scholar/data/score/visual_machine_learning,"<think>\nOkay, let's tackle this query. The us...",NaN,\n\n<ABSTRACT>The article presents a conceptua...,The article presents a conceptual framework fo...,"Yes, the article directly addresses model moni...",85.0
23,visual machine learning,20,visual_machine_learning,https://scholar.google.com.br/scholar?start=20...,https_scholar_google_com_br_scholar_start_20_q...,data/raw_data/visual_machine_learning/https_sc...,https://arxiv.org/abs/1606.05685,Using visual analytics to interpret predictive...,"J Krause, A Perer, E Bertini - arXiv preprint ...",… work with simulated data seems to be a parti...,...,data/articles/visual_machine_learning/https_ar...,data/articles_txt/visual_machine_learning,data/articles_txt/visual_machine_learning/http...,google_scholar/data/score/visual_machine_learning,"<think>\nOkay, let's tackle this query. The us...",NaN,\n\n<ABSTRACT>The article presents techniques ...,The article presents techniques for monitoring...,"Yes, the article directly addresses model moni...",95.0
41,software engineering machine learning,10,software_engineering_machine_learning,https://scholar.google.com.br/scholar?start=10...,https_scholar_google_com_br_scholar_start_10_q...,data/raw_data/software_engineering_machine_lea...,https://arxiv.org/abs/2011.03751,Software engineering for artificial intelligen...,"E Nascimento, A Nguyen-Duc, I Sundbø… - arXiv ...",… has presented several engineering problems t...,...,data/articles/software_engineering_machine_lea...,data/articles_txt/software_engineering_machine...,data/arti

In [52]:
df_searchterms[df_searchterms['score'].isnull()]

,searchterm,start,folder,url,file,path_file,href,title,author_div,description,...,path_article_file,path_txt_folder,path_txt_article,folder_score,file_score,think,file_score_limpo,abstract,assessment,score
90,software engineering ai,130,software_engineering_ai,https://scholar.google.com.br/scholar?start=13...,https_scholar_google_com_br_scholar_start_130_...,data/raw_data/software_engineering_ai/https_sc...,https://arxiv.org/abs/2502.12115,SWE-Lancer: Can Frontier LLMs Earn $1 Million ...,"S Miserendino, M Wang, T Patwardhan… - arXiv p...",… software engineering capabilities and impact...,...,data/articles/software_engineering_ai/https_ar...,data/articles_txt/software_engineering_ai,data/articles_txt/software_engineering_ai/http...,google_scholar/data/score/software_engineering_ai,"<think>\nOkay, let's tackle this query. The us...",NaN,"\n\n{\n ""selected_proposal_id"": 5\n}",NaN,NaN,NaN
509,observability machine learning,130,observability_machine_learning,https://scholar.google.com.br/scholar?start=13...,https_scholar_google_com_br_scholar_start_130_...,data/raw_data/observability_machine_learning/h...,https://ieeexplore.ieee.org/abstract/document/...,Physics-based machine learning with filtering ...,"S Kohtz, P Wang - 2022 Annual Reliability and ...","… of partially observable dynamic systems, whi...",...,data/articles/observability_machine_learning/h...,data/articles_txt/observability_machine_learning,data/articles_txt/observability_machine_learni...,google_scholar/data/score/observability_machin...,"<think>\nOkay, let's tackle this. The user wan...",NaN,\n\n**Abstract:** The paper presents a hybrid ...,NaN,NaN,NaN
572,mlops,80,mlops,https://scholar.google.com.br/scholar?start=80...,https_scholar_google_com_br_scholar_start_80_q...,data/raw_data/mlops/https_scholar_google_com_b...,https://ieeexplore.ieee.org/abstract/document/...,From Development to Deployment: An Approach to...,"A Bodor, M Hnida, D Najima - 2023 14th Interna...",Machine Learning Operations (MLOps) has emerge...,...,data/articles/mlops/https_ieeexplore_ieee_org_...,data/articles_txt/mlops,data/articles_txt/mlops/https_ieeexplore_ieee_...,google_scholar/data/score/mlops,"<think>\nOkay, let's start by reading the prov...",NaN,\n\n**Abstract:** The article proposes an MLOp...,NaN,NaN,NaN


In [51]:
print(df_searchterms[df_searchterms['score'].isnull()]['file_score_limpo'].values[0])



{
  "selected_proposal_id": 5
}


In [34]:
df_searchterms[df_searchterms['score'].isnull()]

,searchterm,start,folder,url,file,path_file,href,title,author_div,description,...,domain_count,path_article_file,path_txt_folder,path_txt_article,folder_score,file_score,score,think,abstract,assessment
52,software engineering machine learning,110,software_engineering_machine_learning,https://scholar.google.com.br/scholar?start=11...,https_scholar_google_com_br_scholar_start_110_...,data/raw_data/software_engineering_machine_lea...,https://link.springer.com/article/10.1007/s105...,How to certify machine learning based safety-c...,"F Tambon, G Laberge, L An, A Nikanjam… - … Sof...",… Machine Learning (ML) is drastically changin...,...,417,data/articles/software_engineering_machine_lea...,data/articles_txt/software_engineering_machine...,data/articles_txt/software_engineering_machine...,google_scholar/data/score/software_engineering...,"<think>\nOkay, let's tackle this query. The us...",NaN,NaN,...,...
90,software engineering ai,130,software_engineering_ai,https://scholar.google.com.br/scholar?start=13...,https_scholar_google_com_br_scholar_start_130_...,data/raw_data/software_engineering_ai/https_sc...,https://arxiv.org/abs/2502.12115,SWE-Lancer: Can Frontier LLMs Earn $1 Million ...,"S Miserendino, M Wang, T Patwardhan… - arXiv p...",… software engineering capabilities and impact...,...,417,data/articles/software_engineering_ai/https_ar...,data/articles_txt/software_engineering_ai,data/articles_txt/software_engineering_ai/http...,google_scholar/data/score/software_engineering_ai,"<think>\nOkay, let's tackle this query. The us...",NaN,NaN,NaN,NaN
509,observability machine learning,130,observability_machine_learning,https://scholar.google.com.br/scholar?start=13...,https_scholar_google_com_br_scholar_start_130_...,data/raw_data/observability_machine_learning/h...,https://ieeexplore.ieee.org/abstract/document/...,Physics-based machine learning with filtering ...,"S Kohtz, P Wang - 2022 Annual Reliability and ...","… of partially observable dynamic systems, whi...",...,381,data/articles/observability_machine_learning/h...,data/articles_txt/observability_machine_learning,data/articles_txt/observability_machine_learni...,google_scholar/data/score/observability_machin...,"<think>\nOkay, let's tackle this. The user wan...",NaN,NaN,NaN,NaN
572,mlops,80,mlops,https://scholar.google.com.br/scholar?start=80...,https_scholar_google_com_br_scholar_start_80_q...,data/raw_data/mlops/https_scholar_google_com_b...,https://ieeexplore.ieee.org/abstract/document/...,From Development to Deployment: An Approach to...,"A Bodor, M Hnida, D Najima - 2023 14th Interna...",Machine Learning Operations (MLOps) has emerge...,...,381,data/articles/mlops/https_ieeexplore_ieee_org_...,data/articles_txt/mlops,data/articles_txt/mlops/https_ieeexplore_ieee_...,google_scholar/data/score/mlops,"<think>\nOkay, let's start by reading the prov...",NaN,NaN,NaN,NaN
